In [1]:
import sys, os
sys.path.append(os.path.abspath('../..'))  # adjust levels to reach project root
from utils.dataframe import load_and_clean_data
df = load_and_clean_data()
df.head()

Processing DataFrame from '1429_1.csv'...
Finished processing '1429_1.csv'. Found 34660 rows.
Processing DataFrame from 'Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products.csv'...
Finished processing 'Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products.csv'. Found 5000 rows.
Processing DataFrame from 'Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products_May19.csv'...
Finished processing 'Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products_May19.csv'. Found 28332 rows.

Merging all datasets...
Total rows after merge: 67992
Dropped 2153 duplicate reviews.
Dropped 6810 rows due to missing (MUST HAVE) data.
Final merged dataset has 59029 rows.


,product_name,categories,review_rating,review_text,review_title,image_urls,review_date,review_recommend,review_helpful_count,source_urls,review_username,review_username,data_source
0,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...","Electronics,iPad & Tablets,All Tablets,Fire Ta...",5.0,This product so far has not disappointed. My c...,Kindle,NaN,2017-01-13T00:00:00.000Z,True,0.0,NaN,Adapter,Adapter,1429_1.csv
1,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...","Electronics,iPad & Tablets,All Tablets,Fire Ta...",5.0,great for beginner or experienced person. Boug...,very fast,NaN,2017-01-13T00:00:00.000Z,True,0.0,NaN,truman,truman,1429_1.csv
2,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...","Electronics,iPad & Tablets,All Tablets,Fire Ta...",5.0,Inexpensive tablet for him to use and learn on...,Beginner tablet for our 9 year old son.,NaN,2017-01-13T00:00:00.000Z,True,0.0,NaN,DaveZ,DaveZ,1429_1.csv
3,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...","Electronics,iPad & Tablets,All Tablets,Fire Ta...",4.0,I've had my Fire HD 8 two weeks now and I love...,Good!!!,NaN,2017-01-13T00:00:00.000Z,True,0.0,NaN,Shacks,Shacks,1429_1.csv
4,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...","Electronics,iPad & Tablets,All Tablets,Fire Ta...",5.0,I bought this for my grand daughter when she c...,Fantastic Tablet for kids,NaN,2017-01-12T00:00:00.000Z,True,0.0,NaN,explore42,explore42,1429_1.csv


In [8]:
!pip install sentence-transformers --user
!pip install transformers torch numpy pandas scikit-learn --user


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [10]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer # NEW: For generating features

# --- Global Variables (Simulating notebook use) ---
# Assuming 'df' is already loaded from the previous cell's output
# If 'df' is not globally available, uncomment the following line and ensure
# load_and_clean_data is accessible.
# from utils.dataframe import load_and_clean_data
# df = load_and_clean_data()

# 1. Text Selection: Use the 'categories' column
TEXT_COLUMN = 'categories'
# MODEL_NAME = 'all-MiniLM-L6-v2' # No longer needed
N_CLUSTERS = 5 # Start with 5 clusters (within your 4-6 range)

# Preprocessing the text data
# Convert to string, fill NaNs, and clean up potential list/quote characters
df['processed_text'] = df[TEXT_COLUMN].astype(str).fillna('unknown product').str.replace(r'[\[\]\'"]', '', regex=True)

# Display a sample of the cleaned data
print(f"Using column for clustering: {TEXT_COLUMN}")
print("\nSample of processed text:")
print(df['processed_text'].head())

Using column for clustering: categories

Sample of processed text:
0    Electronics,iPad & Tablets,All Tablets,Fire Ta...
1    Electronics,iPad & Tablets,All Tablets,Fire Ta...
2    Electronics,iPad & Tablets,All Tablets,Fire Ta...
3    Electronics,iPad & Tablets,All Tablets,Fire Ta...
4    Electronics,iPad & Tablets,All Tablets,Fire Ta...
Name: processed_text, dtype: object


In [11]:
# 2. Feature Generation (TF-IDF)
print("Using TfidfVectorizer to generate numerical features...")

# Initialize TF-IDF Vectorizer
# max_df=0.8 ignores terms that appear in more than 80% of the categories (too common)
# min_df=5 ignores terms that appear in fewer than 5 categories (too rare)
vectorizer = TfidfVectorizer(max_df=0.8, min_df=5, stop_words='english')

# Fit the vectorizer and transform the processed text into features
# We assign the result to 'embeddings' to keep subsequent code blocks the same
embeddings = vectorizer.fit_transform(df['processed_text']).toarray()

print(f"Feature matrix shape: {embeddings.shape}")
print("TF-IDF feature generation complete.")

Using TfidfVectorizer to generate numerical features...
Feature matrix shape: (59029, 225)
TF-IDF feature generation complete.


In [12]:
# 3. Dimensionality Reduction (Using PCA for stability)
N_COMPONENTS = 50 # Reduce to 50 dimensions

print(f"Applying PCA to reduce dimensions to {N_COMPONENTS}...")

# Initialize PCA
# PCA is generally very stable as it relies on simple linear algebra (eigenvectors/values).
reducer = PCA(n_components=N_COMPONENTS, random_state=42)

# Fit and transform the embeddings
embeddings_reduced = reducer.fit_transform(embeddings)

print(f"Reduced embeddings shape: {embeddings_reduced.shape}")
# Optional: Print the explained variance ratio of the first few components
print("Explained Variance of first 5 components:", reducer.explained_variance_ratio_[:5].sum())

Applying PCA to reduce dimensions to 50...
Reduced embeddings shape: (59029, 50)
Explained Variance of first 5 components: 0.845710428139404


In [13]:
# 4. Clustering Algorithm (K-Means)
print(f"Applying K-Means clustering with k = {N_CLUSTERS}...")

# Initialize and train K-Means model
kmeans = KMeans(n_clusters=N_CLUSTERS, random_state=42, n_init=10, verbose=0)
cluster_labels = kmeans.fit_predict(embeddings_reduced)

# Add cluster labels to the DataFrame
df['cluster_id'] = cluster_labels

print("Clustering complete.")
print("\nCluster distribution:")
print(df['cluster_id'].value_counts().sort_index())

Applying K-Means clustering with k = 5...
Clustering complete.

Cluster distribution:
cluster_id
0    14151
1     8244
2    11273
3    19319
4     6042
Name: count, dtype: int64


In [15]:
# 5. Cluster Analysis: Examine the top categories in each cluster

print("--- Cluster Analysis: Top 5 Categories per Cluster ---")

# Iterate through each cluster ID
for cluster_id in sorted(df['cluster_id'].unique()):
    cluster_data = df[df['cluster_id'] == cluster_id]
    
    # Get the top 5 most frequent original category strings
    top_categories = cluster_data[TEXT_COLUMN].value_counts().head(5)
    
    print(f"\nCluster ID {cluster_id} (Count: {len(cluster_data)}):")
    print(top_categories)

# --- Metacategory Assignment (New Column) ---
# Based on the analysis above, you manually define the name for each cluster ID.

# **CUSTOMIZED DICTIONARY BASED ON YOUR CLUSTER OUTPUT:**
metacategory_mapping = {
    # Cluster 0: Fire Tablets, Tablets, Computers & Tablets
    0: "Tablets & E-Readers", 
    
    # Cluster 1: Amazon Echo, Smart Home, Home Automation, Voice Assistants
    1: "Smart Home & Voice Assistants", 
    
    # Cluster 2: AA, AAA, Batteries, Household Batteries
    2: "Batteries & Household Supplies",    
    
    # Cluster 3: Categories unknown - leaving as a placeholder for now.
    3: "Unknown/Mixed Consumer Electronics A",      
    
    # Cluster 4: Categories unknown - leaving as a placeholder for now.
    4: "Unknown/Mixed Consumer Electronics B"      
}

# Add the metacategory column
df['metacategory'] = df['cluster_id'].map(metacategory_mapping)

print("\n--- Final Dataset Preview with Metacategory ---")
print(df[['product_name', 'categories', 'cluster_id', 'metacategory']].head(10))

--- Cluster Analysis: Top 5 Categories per Cluster ---

Cluster ID 0 (Count: 14151):
categories
Fire Tablets,Tablets,Computers & Tablets,All Tablets,Electronics, Tech Toys, Movies, Music,Electronics,iPad & Tablets,Android Tablets,Frys                                                                           10966
Fire Tablets,Learning Toys,Toys,Tablets,Amazon Tablets,Electronics, Tech Toys, Movies, Music,Computers & Tablets,Kids' Tablets,Electronics,iPad & Tablets,Tablets for Kids                                            1676
Computers,Fire Tablets,Electronics Features,Computer Accessories,Tablets,Top Rated,Amazon Tablets,Electronics,Kids' Tablets,iPad & Tablets,Cases & Bags,Electronics, Tech Toys, Movies, Music,Computers & Tablets     1503
Electronics,Computers,Computer Accessories,Cases & Bags,Fire Tablets,Electronics Features,Tablets,Computers & Tablets,Kids' Tablets,Electronics, Tech Toys, Movies, Music,iPad & Tablets,Top Rated                       6
Name: count, dtype: int64

C

In [16]:
print("--- Metacategory Distribution ---")
print(df['metacategory'].value_counts())

--- Metacategory Distribution ---
metacategory
Unknown/Mixed Consumer Electronics A    19319
Tablets & E-Readers                     14151
Batteries & Household Supplies          11273
Smart Home & Voice Assistants            8244
Unknown/Mixed Consumer Electronics B     6042
Name: count, dtype: int64


I have successfully used TF-IDF feature extraction and PCA dimensionality reduction in conjunction with K-Means clustering to automatically group product categories into five distinct and meaningful metacategories, such as "Tablets & E-Readers" and "Batteries & Household Supplies."